# Analysis DATA

In [ ]:
from utilities import *

raw_data = pd.read_excel("data/Datos_Market_copy.xlsx")

sa = SalesAnalysis(raw_data)

data = sa.data

### Create our model

In [ ]:
data_dummies, model = sa.modelization(
    data[sa.brand35], fix_significance=True, interactions=4
)

model.summary()

#### Plot the residuals

In [ ]:
sa.plot_resid_ACF_PACF(model)

In [4]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# Fijar semilla para reproducibilidad
np.random.seed(42)

# Generar datos simulados
data = pd.DataFrame(
    {
        "X1": np.random.randint(1, 10, 100),  # Covariable numérica
        "Factor": np.random.choice(
            ["A", "B", "C"], 100
        ),  # Factor categórico con 3 niveles
        "Y": np.random.randint(1, 20, 100),  # Variable respuesta
    }
)

# Ajustar un modelo lineal con interacción entre X1 y el factor
model = smf.ols(formula="Y ~ X1 * C(Factor) ** 2", data=data).fit()

# Obtener el resumen del modelo
print(model.summary())